# GEO877: Spatial Algorithms

## Practical 1: Points and Spatial Distance Measures
v1, class embedded, sample data, and dataset will load if in same folder as this file

### Problem
A dataset containing 10,000 records with Cartesian (x, y) and WGS84 coordinates (longitude, latitude) and a 'region' classifier is provided.

For all points and for points in each region, calculate the mean and standard deviation of the differences between Cartesian and a spherical distance measures for all pairs of points. Also calculate the maximum absolute error in distance and maximum relative error.

A sample Points class is provided (embedded below) to help get you started...


### Distance functions

#### Euclidean

$$d_{Euclidean}(a, b) = \sqrt{(a_{x} - b_{x})^{2} + (a_{y} - b_{y})^{2} }$$ 

#### Spherical

...

### Pseudocode/Flow diagram
...

### Classes and methods

In [2]:
# class and methods for a geometric point
# =======================================
from numpy import sqrt
import math
class Point():
    # initialise
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y
    
    # representation
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'
        
    # calculate Euclidean distance between two points
    def distEuclidean(self, other):
        return sqrt((self.x-other.x)**2 + (self.y-other.y)**2)
    
    # calculate Manhattan distance between two points
    def distManhattan(self, other):
        return abs(self.x-other.x) + abs(self.y-other.y)

    def distHaversine(self, other):
        my_x_rad, my_y_rad = self.deg2rad(self.x), self.deg2rad(self.y)
        other_x_rad, other_y_rad = self.deg2rad(other.x), self.deg2rad(other.y)
        r = 6371000 # Mittlerer Radius volumengleicher Kugel nach GRS80

        hav_phi = self.haversine_function(other_x_rad - my_x_rad) + math.cos(my_x_rad) * math.cos(other_x_rad) * self.haversine_function(other_y_rad - my_y_rad)
        d = 2 * r * math.asin(sqrt(hav_phi))
        return d

    def deg2rad(self, degree):
        return degree*(math.pi / 180)

    def haversine_function(self, radians):
        return (1 - math.cos(radians))/2

    # Test for equality between Points
    def __eq__(self, other): 
        if not isinstance(other, Point):
            # don't attempt to compare against unrelated types
            return NotImplemented

        return self.x == other.x and self.y == other.y
    # We need this method so that the class will behave sensibly in sets and dictionaries
    def __hash__(self):
        return hash((self.x, self.y))


In [3]:
## Testing Haversine:
bern = (600500, 206750, 47.01180, 7.44521)
zurich = (682000, 252750, 47.42045, 8.52532)

bern_kartesisch = Point(bern[0],bern[1])
zurich_kartesisch = Point(zurich[0], zurich[1])
bern_geographic = Point(bern[2],bern[3])
zurich_geographic = Point(zurich[2],zurich[3])

print(f" Euclidean Distance Zurich Bern = {bern_kartesisch.distEuclidean(zurich_kartesisch)} m")
print(f" Haversine Distance Zurich Bern = {bern_geographic.distHaversine(zurich_geographic)} m")

 Euclidean Distance Zurich Bern = 93585.5223846082 m
 Haversine Distance Zurich Bern = 93378.30307487212 m


### Data

In [4]:
import pandas as pd

# sample data for testing
# -----------------------
# 2 cities with x, y, lat and lng
bern = (600500, 206750, 47.01180, 7.44521)
zurich = (682000, 252750, 47.42045, 8.52532)

# Flickr set of 10K records
# -------------------------
data_folder = "../raw_data/"   # specify folder where dataset is, if different to this file - here a relative path
data_file = "flickr_10000_uk_adm.csv"
input_string = data_folder + data_file
df = pd.read_csv(input_string, sep = ",")

print(df.info()) # Show the data types of the entries in the frame
df[:3] #Output the first three rows of the frame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   photo_id   10000 non-null  int64  
 1   longitude  10000 non-null  float64
 2   latitude   10000 non-null  float64
 3   X          10000 non-null  float64
 4   Y          10000 non-null  float64
 5   adm1_code  10000 non-null  object 
 6   name       10000 non-null  object 
 7   region     10000 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 625.1+ KB
None


,photo_id,longitude,latitude,X,Y,adm1_code,name,region
0,1559,-2.982906,56.456295,339520.5448,729782.1624,GBR-2020,Dundee,Eastern
1,2534,-4.287759,55.867930,256939.5208,666228.5591,GBR-2004,Glasgow,South Western
2,5426,-0.384564,51.828854,511419.5498,215704.3552,GBR-2752,Luton,East


### Solution

In [5]:
# ... goes here
import numpy as np

res_length = (len(df) * (len(df)+1)) // 2 ## Triangular Formula, for 10k rows -> length of ca. 50 Mio

cartesian_distances = np.zeros(res_length)
geographic_distances = np.zeros(res_length)

index = 0

for i, row_a in df.iterrows():
    row_a_cart = Point(row_a["X"], row_a["Y"])
    row_a_geo = Point(row_a["latitude"], row_a["longitude"])

    for j, row_b in df.iloc[i:].iterrows():
        row_b_cart = Point(row_b["X"], row_b["Y"])
        row_b_geo = Point(row_b["latitude"], row_b["longitude"])

        cartesian_distances[index] = row_a_cart.distEuclidean(row_b_cart)
        geographic_distances[index] = row_a_geo.distHaversine(row_b_geo)
        index += 1

In [ ]:
diff = abs(cartesian_distances - geographic_distances)
print(f"the mean difference is: {np.mean(diff)} meter")
print(f"Standard Deviation: {np.std(diff)} meter")
print(f"Max absolute error: {np.max(diff)} meter")

index_max_val = np.argmax(diff)
rel_error_max_absolute = diff[index_max_val] / cartesian_distances[index_max_val] * 100
rel_errors = np.divide(diff, cartesian_distances,out =np.zeros_like(diff), where = cartesian_distances != 0) * 100

print(f"Relative Error of max absolute error: {rel_error_max_absolute}%")
print(f"Max relative Error: {max(rel_errors)}%")

index_max_rel_error = np.argmax(rel_errors)

print(f"absolute distance of max relative Error: {diff[index_max_rel_error]} m")


the mean difference is: 430.7238606746609 meter
Standard Deviation: 443.85605856412224 meter
Max absolute error: 3066.5092711334582 meter
Relative Error of max absolute error: 0.38063064830980836%
Max relative Error: 14.926909268150665%
absolute distance of max relative Error: 0.009030792446724313 m


In [ ]:
# TEST CG